<a href="https://colab.research.google.com/github/thaisbeham/Applied_deep_learning/blob/main/Assigment2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM - Fake News detection

Fake - 0
Real - 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:

import pandas as pd
#from utils import merge_datasets, smaller_set
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import string
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



### Functions

In [19]:
def read_news(path, column_text_name):
  news = pd.read_csv(path)
  news[column_text_name] = news[column_text_name].astype(str)
  if column_text_name != "text":
    news.rename(columns = {column_text_name : "text"}, inplace = True)

  return news

def merge_split_datasets(real, fake, n_testsize = 0.15):
  # take 13000 rows from real data
  #real = real.sample(n = 13000, random_state= 2, axis = 0)

  # add column label
  real['label'] = 1
  fake['label'] = 0

  real = real[['text', 'label']]
  fake = fake[['text', 'label']]
  

  #merge real and fake
  # use only the sentence column on fake
  merged = pd.DataFrame(real.append(fake, ignore_index = True))
  merged = merged.dropna()
   
  merged = merged.sample(frac = 1, random_state= 1, ignore_index = True)#.reset_index()

  merged["text"].dropna(inplace=True)

  # convert the column to string
  merged['text'] = merged['text'].astype(str)

  # apply the `word_tokenize()` function
  tokens = merged['text'].apply(word_tokenize)
  tokens = tokens.astype(str)
  tokens = tokens.apply(str)
  #tokens = [w.apply(str) for w in tokens]
  #merged['text']=merged['text'].fillna('').apply(str)
  print('the type is..', type(tokens))  
  stop_words = set(stopwords.words('english'))
  print(tokens)
  #word_tokens = word_tokenize(tokens)
  
  filtered_sentence = [w for w in tokens if not w.lower() in stop_words]

  X_train, X_test, Y_train, Y_test = train_test_split(filtered_sentence,merged['label'], test_size = n_testsize, random_state = 1)

  #X_test = test['text']
  #Y_test = test['label']

  #create validation set
  #X = train['text']
  #Y = train['label'].astype(int)    
  X_train,X_valid,Y_train,Y_valid = train_test_split(X_train,Y_train,test_size=0.15, random_state=42)
  
  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


def RNN(max_tokens, max_len, dropout):
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_tokens,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(dropout)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

#def LSTM(embedding_vector_features=45):
    
 # model=Sequential()

  #model.add(Embedding(max_words,embedding_vector_features,input_length=max_len))

  #model.add(LSTM(128,activation='relu',return_sequences=True))

  #model.add(Dropout(0.2))

  #model.add(LSTM(128,activation='relu'))

  #model.add(Dropout(0.2))

  # for units in [128,128,64,32]:

  # model.add(Dense(units,activation='relu'))

  # model.add(Dropout(0.2))

  #model.add(Dense(32,activation='relu'))

  #model.add(Dropout(0.2))

  #model.add(Dense(4,activation='softmax'))
  #return model

def predict_result(X_test, Y_test, tok):
  test_sequences = tok.texts_to_sequences(X_test)
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  accr = model.evaluate(test_sequences_matrix,Y_test)
  print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
  predict_labels = model.predict(test_sequences_matrix) 

  # convert the range predicition into 0s or 1s
  Y_pred =[ 1 if i>0.5 else 0 for i in predict_labels  ] 

  print("Number Real news:",sum(Y_pred))
  print("Number Fake news:",len(Y_pred)-sum(Y_pred))
  print()
  print("Classification Report")
  print(classification_report(Y_test, Y_pred, target_names = ['class 0', 'class 1']))
  print()
  print("Confusion Matrix")
  matrix=confusion_matrix(Y_test,Y_pred,labels=[0,1])
  cm=pd.DataFrame(matrix,index=['class_0 pred','class_1 pred'],columns=['class_0 True','class_1 True'])
  print(cm)
  return

def trainer(X_train, Y_train, max_words, max_len, n_batchsize, n_epochs, model):
  tok = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
  tok.fit_on_texts(X_train)
  sequences = tok.texts_to_sequences(X_train)
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
  print('Found %s unique tokens.' % len(sequences_matrix))
  model.summary()
  model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
  model.fit(sequences_matrix,Y_train,batch_size=n_batchsize,epochs=n_epochs,
          validation_split=0.2)
  
  return model, tok



### Read datasets

In [3]:
#Read datasets
fake_news1 = read_news("/content/drive/MyDrive/dados_/fake.csv", "text")
fake_news2 = read_news("/content/drive/MyDrive/dados_/Fake_bisaillon.csv", "text")

real_news1 = read_news("/content/drive/MyDrive/dados_/df_2016.csv", "sentence")
real_news2 = read_news("/content/drive/MyDrive/dados_/True_bisaillon.csv", "text")

#shape of each dataset
print("fake_news1: ", np.shape(fake_news1), "\nfake_news2: ",  np.shape(fake_news2), "\nreal_news1: ",  np.shape(real_news1),"\nreal_news2: ", np.shape(real_news2)  )

real_news1_smallset = real_news1.sample(n = 15000, random_state= 2, axis = 0)

print("real_news1 small set:", np.shape(real_news1_smallset))
# separate the location of the news from the news text (e.g. "WASHINGTON (Reuters)")
real_news2[["loc", "text"]] = real_news2['text'].str.split('-',1, expand = True)
#real_news2.head()





fake_news1:  (12999, 20) 
fake_news2:  (23481, 4) 
real_news1:  (105606, 3) 
real_news2:  (21417, 4)
real_news1 small set: (15000, 3)


### **Train model with 2 datasets with similar lenght**
> Dataset fake.csv and df_2016.csv

In [20]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news1, fake_news1)

the type is.. <class 'pandas.core.series.Series'>
0         ['Elsa', 'Dorfman', 'turned', 'one', 'of', 'th...
1         ['Although', 'it', 'had', 'been', 'alerted', '...
2         ['It', '’', 's', 'not', 'hard', 'to', 'see', '...
3         ['Hackers', 'exposed', 'Clinton', 'as', 'a', '...
4         ['A', 'critical', 'guide', 'to', 'exhibitions'...
                                ...                        
118600    ['A', 'Federal', 'appeals', 'court', 'ruled', ...
118601    ['A', 'longtime', 'travel', 'writer', 'shares'...
118602    ['Vehicles', 'are', 'increasingly', 'able', 't...
118603    ['Old-fashioned', 'gets', 'radical', 'in', 'a'...
118604    ['After', 'a', '2-3', 'start', ',', 'Ben', 'Mc...
Name: text, Length: 118605, dtype: object


#### Parameters tunning

In [22]:
max_words = 1000
max_len = 300
n_batchsize = 128
n_epochs = 10
dropout = 0.2

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("stopwords")
def get_frequency(text):
        lemmatizer = WordNetLemmatizer()
        #text = tuple
        #text = [x for x in tuple]
        text = ''.join(text)
        trueCauseWords = nltk.word_tokenize(text)
        stopword = set(stopwords.words("english"))
        words = [
                lemmatizer.lemmatize(word).lower()
                for word in trueCauseWords
                if re.match("\w", word) and (word.lower() not in stopword)
                ]
        #trueCauseWords = [word for line in trueCauseText for word in line.split()]
        counts = Counter( x for x in words)
        print(counts.most_common(15))
        return

In [ ]:
get_frequency(X_train)

#### Train

In [23]:
model = RNN(max_tokens = max_words, max_len = max_len, dropout = dropout )
from keras.utils import plot_model
plot_model(model, to_file='model.png')
trained_model, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model)

Found 85691 unique tokens.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None

#### Predict - Valid set

In [24]:
predict_result(X_valid, Y_valid, tok)

473/473 [==============================] - 3s 6ms/step - loss: 0.0257 - accuracy: 0.9934
Test set
  Loss: 0.026
  Accuracy: 0.993
473/473 [==============================] - 3s 5ms/step
Number Real news: 13496
Number Fake news: 1627

Classification Report
              precision    recall  f1-score   support

     class 0       0.98      0.96      0.97      1665
     class 1       0.99      1.00      1.00     13458

    accuracy                           0.99     15123
   macro avg       0.99      0.98      0.98     15123
weighted avg       0.99      0.99      0.99     15123


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          1596            69
class_1 pred            31         13427


In [ ]:
### Test other parameters

max_words = 2000
max_len = 500 
n_batchsize = 128
n_epochs = 10
dropout = 0.2

model2 = RNN(max_tokens = max_words, max_len = max_len, dropout = dropout )

trained_model2, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model2)
predict_result(X_valid, Y_valid, tok)

The second parameter test has higher valeus for vocabulary size ( max_words) and lenght of sentences (max_len). This choice resulted in even higher accuracy, which could indicate overfit of the data, therefore the previous parameters were the one choosen.

In [ ]:
#test other model
https://www.kaggle.com/code/khotijahs1/using-lstm-for-nlp-text-classification
model2 = LSTM()

trained_model2, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model2)
predict_result(X_valid, Y_valid, tok)

In [ ]:
parmeters = {max_words = [500, 1000, 2000],
            max_len = [100, 500, 1000],
            dropout = [0.1, 0.2, 0.5] }
def CVsearch(parameters, model):
  max_words = [parameter[0] for parameter in parameters]
  max_len =  [parameter[1] for parameter in parameters]
  dropout = [parameter[2] for parameter in parameters]

  optimal_fscore = 0  
  optimal_params = [0,0,0,0]

  for i in max_words:
    for j in max_len:
      for k in dropout:

        print("checking parameters", output_dim,input_dim, emb, hidd)
        kf = k_fold(n_splits=folds)
        indices = kf.split()
       








Predict - Test set

In [ ]:
predict_result(X_test, Y_test, tok)

### **Train model with 2 datasets**

> Dataset fake.csv and df_2016.csv


In [ ]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news1, fake_news1)

#### Parameters tunning

In [ ]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 10

#### Train

In [ ]:
model = RNN()

trained_model, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model)

#### Predict - Valid set

In [ ]:
predict_result(X_valid, Y_valid, tok)

#### Predict - Test set

In [ ]:
predict_result(X_test, Y_test, tok)

### **Train model with  4 datasets**

> Datasets: fake.csv, df_2016.csv, fake_Bisallion.csv and True_Bisallion.csv

In [ ]:

real_news = pd.DataFrame(real_news1["text"].append(real_news2["text"], ignore_index = True))
fake_news = pd.DataFrame(fake_news1["text"].append(fake_news2["text"], ignore_index = True))
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news, fake_news)

#### Parameters tunning

In [ ]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 10


#### Train model

In [ ]:
model = RNN()

trained_model, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model)

#### Predict - Valid set

In [ ]:
predict_result(X_valid, Y_valid, tok)

#### Predict - test set

In [ ]:
predict_result(X_test, Y_test, tok)